In [11]:
using CoolProp

"""
    calculate_cv_2phase(fluid::String, T::Real, x::Real)

Calculate the two-phase specific heat capacity at constant volume for a given fluid.

Parameters:
- fluid: String name of the fluid (e.g., "Hydrogen")
- T: Temperature in K
- x: Quality (vapor fraction) between 0 and 1

Returns:
- cv_2p: Two-phase specific heat capacity at constant volume [J/kg-K]
"""
function calculate_cv_2phase(fluid::String, T::Real, x::Real)
    # Input validation
    if x < 0 || x > 1
        throw(ArgumentError("Quality must be between 0 and 1"))
    end
    
    # Create the fluid string for CoolProp
    fluid_str = fluid
    
    # Get saturated properties at temperature T
    Psat = PropsSI("P", "T", T, "Q", 0, fluid_str)  # Saturation pressure
    
    # Get liquid and vapor specific heats
    cv_l = PropsSI("CVMASS", "T", T, "Q", 0, fluid_str)  # Saturated liquid Cv
    cv_v = PropsSI("CVMASS", "T", T, "Q", 1, fluid_str)  # Saturated vapor Cv
    
    # Get specific volumes
    rho_l = PropsSI("D", "T", T, "Q", 0, fluid_str)  # Saturated liquid density
    rho_v = PropsSI("D", "T", T, "Q", 1, fluid_str)  # Saturated vapor density
    v_l = 1/rho_l
    v_v = 1/rho_v
    v_fg = v_v - v_l
    
    # Get enthalpies
    h_l = PropsSI("H", "T", T, "Q", 0, fluid_str)  # Saturated liquid enthalpy
    h_v = PropsSI("H", "T", T, "Q", 1, fluid_str)  # Saturated vapor enthalpy
    h_fg = h_v - h_l
    
    # Calculate dhfg/dT using finite difference
    δT = 0.1  # Small temperature difference for derivative
    h_fg_plus = PropsSI("H", "T", T + δT, "Q", 1, fluid_str) - 
                PropsSI("H", "T", T + δT, "Q", 0, fluid_str)
    dhfg_dT = (h_fg_plus - h_fg) / δT
    
    # Calculate two-phase specific heat
    cv_2p = (1-x)*cv_l + x*cv_v #+ x*(1-x)*(dhfg_dT^2)*T/(Psat*v_fg)
    
    return cv_2p
end;

In [12]:
# Example usage
T = 20.0  # Temperature in K
x = 0.5   # Quality (vapor fraction)
cv_2p = calculate_cv_2phase("ParaHydrogen", T, x)
println("Two-phase Cv at T = $T K and x = $x: $cv_2p J/kg-K")

Two-phase Cv at T = 20.0 K and x = 0.5: 6043.472903681015 J/kg-K


In [13]:
cv = PropsSI("CVMASS", "T", T, "Q", x, "ParaHydrogen")
print(cv)

7066.31499560025